In [1]:
import numpy as np
import pandas as pd
import os
from pathlib import Path
import matplotlib as plt
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from scipy.ndimage import shift
from IPython.display import clear_output
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import OrdinalEncoder
import seaborn
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.svm import SVR
from sklearn.svm import LinearSVR
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import PolynomialFeatures

In [2]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/house-prices-advanced-regression-techniques/sample_submission.csv
/kaggle/input/house-prices-advanced-regression-techniques/data_description.txt
/kaggle/input/house-prices-advanced-regression-techniques/train.csv
/kaggle/input/house-prices-advanced-regression-techniques/test.csv


In [3]:
train_data = pd.read_csv(Path("/kaggle/input/house-prices-advanced-regression-techniques/train.csv"))
test_data = pd.read_csv(Path("/kaggle/input/house-prices-advanced-regression-techniques/test.csv"))
sample_submission = pd.read_csv(Path("/kaggle/input/house-prices-advanced-regression-techniques/sample_submission.csv"))
train_data = train_data.drop("Id",axis=1)
test_data = test_data.drop("Id",axis=1)

In [4]:
X_train_data = train_data.drop("SalePrice",axis=1)
y_train_data = train_data["SalePrice"]

In [5]:
X_train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 79 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   MSSubClass     1460 non-null   int64  
 1   MSZoning       1460 non-null   object 
 2   LotFrontage    1201 non-null   float64
 3   LotArea        1460 non-null   int64  
 4   Street         1460 non-null   object 
 5   Alley          91 non-null     object 
 6   LotShape       1460 non-null   object 
 7   LandContour    1460 non-null   object 
 8   Utilities      1460 non-null   object 
 9   LotConfig      1460 non-null   object 
 10  LandSlope      1460 non-null   object 
 11  Neighborhood   1460 non-null   object 
 12  Condition1     1460 non-null   object 
 13  Condition2     1460 non-null   object 
 14  BldgType       1460 non-null   object 
 15  HouseStyle     1460 non-null   object 
 16  OverallQual    1460 non-null   int64  
 17  OverallCond    1460 non-null   int64  
 18  YearBuil

In [6]:
test_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1459 entries, 0 to 1458
Data columns (total 79 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   MSSubClass     1459 non-null   int64  
 1   MSZoning       1455 non-null   object 
 2   LotFrontage    1232 non-null   float64
 3   LotArea        1459 non-null   int64  
 4   Street         1459 non-null   object 
 5   Alley          107 non-null    object 
 6   LotShape       1459 non-null   object 
 7   LandContour    1459 non-null   object 
 8   Utilities      1457 non-null   object 
 9   LotConfig      1459 non-null   object 
 10  LandSlope      1459 non-null   object 
 11  Neighborhood   1459 non-null   object 
 12  Condition1     1459 non-null   object 
 13  Condition2     1459 non-null   object 
 14  BldgType       1459 non-null   object 
 15  HouseStyle     1459 non-null   object 
 16  OverallQual    1459 non-null   int64  
 17  OverallCond    1459 non-null   int64  
 18  YearBuil

In [7]:
X_train_data.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal
1,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,...,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal
2,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,0,NaN,NaN,NaN,0,9,2008,WD,Normal
3,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,...,0,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml
4,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,...,0,0,NaN,NaN,NaN,0,12,2008,WD,Normal


**Prepairing the data :**

In [8]:
imputer = SimpleImputer(strategy="most_frequent")
imputer.fit(pd.concat([X_train_data,test_data]))

SimpleImputer(strategy='most_frequent')

In [9]:
c = X_train_data.columns
i_train = X_train_data.index
i_test = test_data.index
X_train_data = pd.DataFrame(imputer.transform(X_train_data),columns = c,index = i_train)
test_data = pd.DataFrame(imputer.transform(test_data),columns = c,index=i_test)

In [10]:
 str_cat = [ "MSZoning"      ,
  "Street"        ,
  "Alley"         ,
  "LotShape"      ,
  "LandContour"   ,
  "Utilities"     ,
  "LotConfig"     ,
  "LandSlope"     ,
  "Neighborhood"  ,
  "Condition1"    ,
  "Condition2"    ,
  "BldgType"      ,
  "HouseStyle"    ,
  "RoofStyle"     ,
  "RoofMatl"      ,
  "Exterior1st"   ,
  "Exterior2nd"   ,
  "MasVnrType"    ,
  "ExterQual"     ,
  "ExterCond"     ,
  "Foundation"    ,
  "BsmtQual"      ,
  "BsmtCond"      ,
  "BsmtExposure"  ,
  "BsmtFinType1"  ,
  "BsmtFinType2"  ,
  "Heating"       ,
  "HeatingQC"     ,
  "CentralAir"    ,
  "Electrical"    ,
  "KitchenQual"   ,
  "Functional"    ,
  "FireplaceQu"   ,
  "GarageType"    ,
  "GarageFinish"  ,
  "GarageQual"    ,
  "GarageCond"    ,
  "PavedDrive"    ,
  "PoolQC"        ,
  "Fence"         ,
  "MiscFeature"   ,
  "SaleCondition" ,
  "SaleType"      ]

In [11]:
def OneHotE(dataframe,cat_col,cat_encoder):
    dataframe_num = dataframe.drop(cat_col,axis=1)
    dataframe_cat = dataframe[cat_col]
    dataframe_cat = cat_encoder.transform(dataframe_cat)
    dataframe_cat = pd.DataFrame(dataframe_cat.toarray(),columns=cat_encoder.get_feature_names_out(),index=dataframe.index)
    dataframe = pd.concat([dataframe_num,dataframe_cat],axis=1)    
    return dataframe

In [12]:
cat_encoder = OneHotEncoder()

In [13]:
cat_encoder.fit(pd.concat([X_train_data[str_cat],test_data[str_cat]]))

OneHotEncoder()

In [14]:
X_train_data = OneHotE(X_train_data,str_cat,cat_encoder)
test_data = OneHotE(test_data,str_cat,cat_encoder)

In [15]:
scaler = StandardScaler()
for column in X_train_data.columns:
    X_train_data[column] = scaler.fit_transform(np.array(X_train_data[column]).reshape(-1,1))

In [16]:
poly_features = PolynomialFeatures(degree=2,include_bias=False)
X_train_data_poly =  poly_features.fit_transform(X_train_data)

In [17]:
X_train_data_poly = pd.DataFrame(X_train_data_poly,index = X_train_data.index , columns = poly_features.get_feature_names_out(X_train_data.columns))

In [18]:
X_train_data_poly.head()

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,SaleType_ConLw^2,SaleType_ConLw SaleType_New,SaleType_ConLw SaleType_Oth,SaleType_ConLw SaleType_WD,SaleType_New^2,SaleType_New SaleType_Oth,SaleType_New SaleType_WD,SaleType_Oth^2,SaleType_Oth SaleType_WD,SaleType_WD^2
0,0.073375,-0.146189,-0.207142,0.651479,-0.517200,1.050994,0.878668,0.514104,0.575425,-0.288653,...,0.003436,0.017701,0.00266,-0.022879,0.091181,0.013702,-0.117853,0.002059,-0.01771,0.152328
1,-0.872563,0.524992,-0.091886,-0.071836,2.179628,0.156734,-0.429577,-0.570750,1.171992,-0.288653,...,0.003436,0.017701,0.00266,-0.022879,0.091181,0.013702,-0.117853,0.002059,-0.01771,0.152328
2,0.073375,-0.011953,0.073480,0.651479,-0.517200,0.984752,0.830215,0.325915,0.092907,-0.288653,...,0.003436,0.017701,0.00266,-0.022879,0.091181,0.013702,-0.117853,0.002059,-0.01771,0.152328
3,0.309859,-0.369915,-0.096897,0.651479,-0.517200,-1.863632,-0.720298,-0.570750,-0.499274,-0.288653,...,0.003436,0.017701,0.00266,-0.022879,0.091181,0.013702,-0.117853,0.002059,-0.01771,0.152328
4,0.073375,0.703973,0.375148,1.374795,-0.517200,0.951632,0.733308,1.366489,0.463568,-0.288653,...,0.003436,0.017701,0.00266,-0.022879,0.091181,0.013702,-0.117853,0.002059,-0.01771,0.152328


**Training a model:**

In [19]:
pipeline = Pipeline([("classifier",LinearRegression())])

In [20]:
param_grid = [{"classifier__fit_intercept":[True,False]}] 

In [21]:
grid_search = GridSearchCV(pipeline,param_grid,cv=3,scoring="r2")

In [22]:
grid_search.fit(X_train_data,y_train_data)

GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('classifier', LinearRegression())]),
             param_grid=[{'classifier__fit_intercept': [True, False]}],
             scoring='r2')

In [23]:
grid_search.best_params_


{'classifier__fit_intercept': True}

In [24]:
cv_res = pd.DataFrame(grid_search.cv_results_)
cv_res.sort_values(by="mean_test_score", ascending=False, inplace=True)
cv_res.head()

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_classifier__fit_intercept,params,split0_test_score,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score
0,0.110026,0.018369,0.025138,0.002001,True,{'classifier__fit_intercept': True},-6.722176e+24,-8.903528e+25,-1.262500e+21,-3.191957e+25,4.048000e+25,1
1,0.147687,0.001203,0.025451,0.002741,False,{'classifier__fit_intercept': False},-4.167458e+25,-7.241799e+21,-1.608103e+26,-6.749736e+25,6.813964e+25,2


**Prediction :**

In [25]:
final_evaluation = grid_search.predict(test_data)

In [26]:
d = {'Id':np.arange(len(final_evaluation)) + 1461,'SalePrice':final_evaluation}
df = pd.DataFrame(data=d)
df

,Id,SalePrice
0,1461,2.214269e+19
1,1462,5.630712e+19
2,1463,5.696008e+19
3,1464,5.533491e+19
4,1465,5.082372e+19
...,...,...
1454,2915,3.446998e+19
1455,2916,3.554393e+19
1456,2917,5.394381e+19
1457,2918,3.951578e+19


In [27]:
df.to_csv('out.csv',index=False)